## Import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from os import path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from math import acos, cos, radians, sin
print(os.listdir("../input"))

['train.csv', 'sample_submission.csv', 'test.csv']


## Data Loading

In [2]:
PATH = "../input"
df = pd.read_csv(PATH + '/train.csv')
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [3]:
df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null datetime64[ns]
dropoff_datetime      1458644 non-null datetime64[ns]
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(3), object(2)
memory usage: 122.4+ MB


> ## Form calcul distance

In [5]:
def distance(latitude1, longitude1, latitude2, longitude2):
    return (6366 * acos(
    cos(radians(latitude1)) *
    cos(radians(latitude2)) *
    cos(radians(longitude2) - radians(longitude1)) +
    sin(radians(latitude1)) *
    sin(radians(latitude2))   
    ))

## Split

In [6]:
pickup_latitude = df['pickup_latitude']
pickup_longitude = df['pickup_longitude']
dropoff_latitude = df['dropoff_latitude']
dropoff_longitude = df['dropoff_longitude']
test = []

for la1, lo1, la2, lo2 in zip(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude):
    test = distance(la1, lo1, la2, lo2)

ValueError: math domain error

In [36]:
df['year_pickup_datetime'] = df['pickup_datetime'].dt.year
df['month_pickup_datetime'] = df['pickup_datetime'].dt.month
df['day_pickup_datetime'] = df['pickup_datetime'].dt.day
df['hour_pickup_datetime'] = df['pickup_datetime'].dt.hour
df['minute_pickup_datetime'] = df['pickup_datetime'].dt.minute
df['seconde_pickup_datetime'] = df['pickup_datetime'].dt.minute * 60
df['weekday_pickup_datetime'] = df['pickup_datetime'].dt.weekday
df['distance'] = test

df_filter = (df['passenger_count'] > 0) & (df['trip_duration'] > 120) & (df['trip_duration'] < 10800)
df_result = df.loc[df_filter]
df_result.shape

(1428201, 19)

In [37]:
columns = ['passenger_count', 'vendor_id', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'year_pickup_datetime', 'month_pickup_datetime', 
          'day_pickup_datetime', 'hour_pickup_datetime', 'minute_pickup_datetime', 'seconde_pickup_datetime', 'weekday_pickup_datetime','distance']
X = df_result[columns]
y = df_result['trip_duration']
X.shape, y.shape

((1428201, 14), (1428201,))

## Modeling

In [38]:
rf = RandomForestRegressor()
rs = ShuffleSplit(n_splits=3, test_size=.7, random_state=42)
#rs.get_n_splits(rf, X, y)
score = -cross_val_score(rf, X, y, cv=rs, scoring='neg_mean_squared_log_error')
np.sqrt(score.mean())

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.35667318578205653

In [39]:
rf.fit(X, y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

## Data_test

In [40]:
df_test = pd.read_csv(PATH + '/test.csv')
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])
df_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [41]:
colomns_test = ['passenger_count', 'vendor_id', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'year_pickup_datetime', 'month_pickup_datetime', 
          'day_pickup_datetime', 'hour_pickup_datetime', 'minute_pickup_datetime', 'seconde_pickup_datetime', 'weekday_pickup_datetime','distance']
df_test['year_pickup_datetime'] = df_test['pickup_datetime'].dt.year
df_test['month_pickup_datetime'] = df_test['pickup_datetime'].dt.month
df_test['day_pickup_datetime'] = df_test['pickup_datetime'].dt.day
df_test['hour_pickup_datetime'] = df_test['pickup_datetime'].dt.hour
df_test['minute_pickup_datetime'] = df_test['pickup_datetime'].dt.minute
df_test['seconde_pickup_datetime'] = df_test['pickup_datetime'].dt.minute * 60
df_test['weekday_pickup_datetime'] = df_test['pickup_datetime'].dt.weekday
df_test['distance'] = df['distance']
df_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,year_pickup_datetime,month_pickup_datetime,day_pickup_datetime,hour_pickup_datetime,minute_pickup_datetime,seconde_pickup_datetime,weekday_pickup_datetime,distance
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,2016,6,30,23,59,3540,3,0.281799
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,2016,6,30,23,59,3540,3,0.281799
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,2016,6,30,23,59,3540,3,0.281799
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,2016,6,30,23,59,3540,3,0.281799
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,2016,6,30,23,59,3540,3,0.281799


In [42]:
X_test = df_test[colomns_test]

In [43]:
y_pred = rf.predict(X_test)
y_pred.mean()

853.5750698250293

## Data_Submission

In [44]:
submission = pd.read_csv(PATH + '/sample_submission.csv')
submission.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [45]:
submission['trip_duration'] = y_pred
submission.head()

,id,trip_duration
0,id3004672,733.1
1,id3505355,572.7
2,id1217141,449.7
3,id2150126,1317.2
4,id1598245,273.6


In [46]:
submission.describe()

,trip_duration
count,625134.000000
mean,853.575070
std,594.694252
min,132.700000
25%,446.300000
50%,689.000000
75%,1071.100000
max,6394.900000


## Post_Data

In [48]:
submission.to_csv('result.csv', index=False)